In [103]:
import pandas as pd
import numpy as np

In [143]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression

In [234]:
from sklearn.model_selection import KFold,cross_val_score,GridSearchCV
from sklearn.preprocessing import StandardScaler

In [436]:
from sklearn.preprocessing import LabelEncoder

# Градиентный бустинг

#### 1 Считайте таблицу с признаками из файла features.csv с помощью кода, приведенного выше. Удалите признаки, связанные с итогами матча (они помечены в описании данных как отсутствующие в тестовой выборке).

In [92]:
data=pd.read_csv("features.csv",index_col='match_id')
data.head()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time,duration,radiant_win,tower_status_radiant,tower_status_dire,barracks_status_radiant,barracks_status_dire
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,4,2,2,-52.0,2874,1,1796,0,51,0
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,4,3,1,-5.0,2463,1,1974,0,63,1
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,4,3,1,13.0,2130,0,0,1830,0,63
3,1430263531,1,29,4,1779,1056,14,0,0,5,...,4,2,0,27.0,1459,0,1920,2047,50,63
4,1430282290,7,13,4,1431,1090,8,1,0,8,...,3,3,0,-16.0,2449,0,4,1974,3,63


In [93]:
df=data.drop(["duration",
           "radiant_win",
           "tower_status_radiant",
           "tower_status_dire",
           "barracks_status_dire",
           "barracks_status_radiant"],axis=1)

#### 2 Проверьте выборку на наличие пропусков с помощью функции count(), которая для каждого столбца показывает число заполненных значений. Много ли пропусков в данных? Запишите названия признаков, имеющих пропуски, и попробуйте для любых двух из них дать обоснование, почему их значения могут быть пропущены.

In [94]:
df.count().head(20)

start_time    97230
lobby_type    97230
r1_hero       97230
r1_level      97230
r1_xp         97230
r1_gold       97230
r1_lh         97230
r1_kills      97230
r1_deaths     97230
r1_items      97230
r2_hero       97230
r2_level      97230
r2_xp         97230
r2_gold       97230
r2_lh         97230
r2_kills      97230
r2_deaths     97230
r2_items      97230
r3_hero       97230
r3_level      97230
dtype: int64

In [96]:
arr=df.count()
NANcolumns=[x[0] for x in arr.iteritems() if x[1]!=97230]
NANcolumns

['first_blood_time',
 'first_blood_team',
 'first_blood_player1',
 'first_blood_player2',
 'radiant_bottle_time',
 'radiant_courier_time',
 'radiant_flying_courier_time',
 'radiant_first_ward_time',
 'dire_bottle_time',
 'dire_courier_time',
 'dire_flying_courier_time',
 'dire_first_ward_time']

#### 3 Замените пропуски на нули с помощью функции fillna(). На самом деле этот способ является предпочтительным для логистической регрессии, поскольку он позволит пропущенному значению не вносить никакого вклада в предсказание. Для деревьев часто лучшим вариантом оказывается замена пропуска на очень большое или очень маленькое значение — в этом случае при построении разбиения вершины можно будет отправить объекты с пропусками в отдельную ветвь дерева. Также есть и другие подходы — например, замена пропуска на среднее значение признака. Мы не требуем этого в задании, но при желании попробуйте разные подходы к обработке пропусков и сравните их между собой. 

In [102]:
df.fillna(value=0,inplace=True)

#### 4 Какой столбец содержит целевую переменную? Запишите его название.

In [ ]:
radiant_win

#### 5 Забудем, что в выборке есть категориальные признаки, и попробуем обучить градиентный бустинг над деревьями на имеющейся матрице "объекты-признаки". Зафиксируйте генератор разбиений для кросс-валидации по 5 блокам (KFold), не забудьте перемешать при этом выборку (shuffle=True), поскольку данные в таблице отсортированы по времени, и без перемешивания можно столкнуться с нежелательными эффектами при оценивании качества. Оцените качество градиентного бустинга (GradientBoostingClassifier) с помощью данной кросс-валидации, попробуйте при этом разное количество деревьев (как минимум протестируйте следующие значения для количества деревьев: 10, 20, 30). Долго ли настраивались классификаторы? Достигнут ли оптимум на испытанных значениях параметра n_estimators, или же качество, скорее всего, продолжит расти при дальнейшем его увеличении?

In [146]:
data=pd.read_csv("features.csv",index_col='match_id')
data.drop(["duration",
           "tower_status_radiant",
           "tower_status_dire",
           "barracks_status_dire",
           "barracks_status_radiant"],axis=1,inplace=True)
data.fillna(value=0,inplace=True)
df=data.copy()

In [185]:
gb=GradientBoostingClassifier(n_estimators=30,random_state=241)

In [186]:
kf=KFold(n_splits=5,random_state=241,shuffle=True)

In [187]:
x=df.iloc[:,:-1]
y=df.iloc[:,-1]
%time np.average(cross_val_score(gb,X=x,y=y,cv=kf,scoring='roc_auc')) # 30 деревьев

CPU times: user 1min 13s, sys: 626 ms, total: 1min 13s
Wall time: 1min 13s


0.6894962060591201

In [202]:
gb=GradientBoostingClassifier(n_estimators=40,random_state=241)
np.average(cross_val_score(gb,X=x,y=y,cv=kf,scoring='roc_auc')) # 40 деревьев

CPU times: user 1min 48s, sys: 1.36 s, total: 1min 49s
Wall time: 1min 51s


0.69413112147303369

### Отчет

#### 1 Какие признаки имеют пропуски среди своих значений? Что могут означать пропуски в этих признаках (ответьте на этот вопрос для двух любых признаков)?

Пропуски присутствуют в следующих столбцах:

 'first_blood_time',
 'first_blood_team',
 'first_blood_player1',
 'first_blood_player2',
Если событие "первая кровь" не успело произойти за первые 5 минут, то признаки принимают пропущенное значение
 
 
 'radiant_bottle_time',
 'radiant_courier_time',
 'radiant_flying_courier_time',
 'radiant_first_ward_time',
 'dire_bottle_time',
 'dire_courier_time',
 'dire_flying_courier_time',
 'dire_first_ward_time'
Поля, связанные с теми или иными приобретенными предметами, соответственно если предмет не был приобретен командой, то поле пустое

#### 2 Как называется столбец, содержащий целевую переменную?

radiant_win

#### 3 Как долго проводилась кросс-валидация для градиентного бустинга с 30 деревьями? Инструкцию по измерению времени можно найти ниже по тексту. Какое качество при этом получилось? Напомним, что в данном задании мы используем метрику качества AUC-ROC.

Wall time: 1min 13s
Качество на кросс-валидации по 5 блокам c 30 деревьями: 0.6894962060591201

#### 4 Имеет ли смысл использовать больше 30 деревьев в градиентном бустинге? Что бы вы предложили делать, чтобы ускорить его обучение при увеличении количества деревьев?

Увеличение количества деревьев ведет к улучшению качества предсказания, но может вызвать переобучение модели

Для того, чтобы ускорить обучение можно сделать то, что сказанно в "Рекомендации и советы". Тоесть уменьшить объем выборки и/или уменьшить глубину деревьев

# Логистическая регрессия

#### 1 Оцените качество логистической регрессии (sklearn.linear_model.LogisticRegression с L2-регуляризацией) с помощью кросс-валидации по той же схеме, которая использовалась для градиентного бустинга. Подберите при этом лучший параметр регуляризации (C). Какое наилучшее качество у вас получилось? Как оно соотносится с качеством градиентного бустинга? Чем вы можете объяснить эту разницу? Быстрее ли работает логистическая регрессия по сравнению с градиентным бустингом?

In [232]:
bacup=df.copy()

In [237]:
x.head()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,0,35.0,103.0,-84.0,221.0,3,4,2,2,-52.0
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,0,-20.0,149.0,-84.0,195.0,5,4,3,1,-5.0
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,1,-39.0,45.0,-77.0,221.0,3,4,3,1,13.0
3,1430263531,1,29,4,1779,1056,14,0,0,5,...,0,-30.0,124.0,-80.0,184.0,0,4,2,0,27.0
4,1430282290,7,13,4,1431,1090,8,1,0,8,...,0,46.0,182.0,-80.0,225.0,6,3,3,0,-16.0


In [253]:
x=df.iloc[:,:-1]
y=df.iloc[:,-1]

In [254]:
scaler=StandardScaler() #Для маштабирования данных
x=scaler.fit_transform(x)

In [257]:
lr=LogisticRegression(penalty='l2',random_state=241,C=2.8947368421052628)
C={"C":np.linspace(1,10,20)}
grid=GridSearchCV(lr,param_grid=C,scoring='roc_auc',cv=kf)

In [256]:
# Подбор наилучшего С
grid.fit(X=x,y=y)
grid.best_estimator_.C

2.8947368421052628

In [258]:
%time np.average(cross_val_score(lr,X=x,y=y,cv=kf,scoring='roc_auc')) 

CPU times: user 12.3 s, sys: 563 ms, total: 12.9 s
Wall time: 12.2 s


0.71630660692837989

#### 2 Среди признаков в выборке есть категориальные, которые мы использовали как числовые, что вряд ли является хорошей идеей. Категориальных признаков в этой задаче одиннадцать: lobby_type и r1_hero, r2_hero, ..., r5_hero, d1_hero, d2_hero, ..., d5_hero. Уберите их из выборки, и проведите кросс-валидацию для логистической регрессии на новой выборке с подбором лучшего параметра регуляризации. Изменилось ли качество? Чем вы можете это объяснить?

In [266]:
df2=df.drop(["lobby_type",
             "r1_hero",
             "r2_hero",
             "r3_hero",
             "r4_hero",
             "r5_hero",
             "d1_hero",
             "d2_hero",
             "d3_hero",
             "d4_hero",
             "d5_hero"],axis=1)

In [267]:
x=df2.iloc[:,:-1]
y=df2.iloc[:,-1]
x=scaler.fit_transform(x)

In [271]:
lr=LogisticRegression(penalty='l2',random_state=241,C=1.4736842105263157)

In [269]:
# Подбор наилучшего С
grid.fit(X=x,y=y)
grid.best_estimator_.C

1.4736842105263157

In [272]:
%time np.average(cross_val_score(lr,X=x,y=y,cv=kf,scoring='roc_auc')) 

CPU times: user 10.5 s, sys: 452 ms, total: 10.9 s
Wall time: 10.2 s


0.71637077592651877

#### 3 На предыдущем шаге мы исключили из выборки признаки rM_hero и dM_hero, которые показывают, какие именно герои играли за каждую команду. Это важные признаки — герои имеют разные характеристики, и некоторые из них выигрывают чаще, чем другие. Выясните из данных, сколько различных идентификаторов героев существует в данной игре (вам может пригодиться фукнция unique или value_counts).

In [277]:
df3=df[["r1_hero",
        "r2_hero",
        "r3_hero",
        "r4_hero",
        "r5_hero",
        "d1_hero",
        "d2_hero",
        "d3_hero",
        "d4_hero",
        "d5_hero"]]  

In [418]:
df3=df3.astype(object)

In [419]:
df3.describe()

,r1_hero,r2_hero,r3_hero,r4_hero,r5_hero,d1_hero,d2_hero,d3_hero,d4_hero,d5_hero
count,97230,97230,97230,97230,97230,97230,97230,97230,97230,97230
unique,108,108,108,108,108,108,108,108,108,108
top,11,7,7,7,7,11,7,7,7,7
freq,5045,4201,4235,4564,4552,4030,4106,4223,4408,4321


In [406]:
#Проверка на то, что в каком-лобо столбце нет значения не встречающегося в других столбцах
tmp=df3["r1_hero"].unique()
tmp.sort()
for i in df3.columns:
    tmp2=df3[i].unique()
    tmp2.sort()
    if(tmp2.all()==tmp.all()):
        print ("Ok")
        tmp=tmp2
    else:
        print ("ERR")

Ok
Ok
Ok
Ok
Ok
Ok
Ok
Ok
Ok
Ok


#### 4 Воспользуемся подходом "мешок слов" для кодирования информации о героях. Пусть всего в игре имеет N различных героев. Сформируем N признаков, при этом i-й будет равен нулю, если i-й герой не участвовал в матче; единице, если i-й герой играл за команду Radiant; минус единице, если i-й герой играл за команду Dire. Ниже вы можете найти код, который выполняет данной преобразование. Добавьте полученные признаки к числовым, которые вы использовали во втором пункте данного этапа.

In [645]:
data=pd.read_csv("features.csv")
data.drop(["duration",
           "tower_status_radiant",
           "tower_status_dire",
           "barracks_status_dire",
           "barracks_status_radiant",
           "lobby_type"],axis=1,inplace=True)
data.fillna(value=0,inplace=True)

In [646]:
hero=["r2_hero","r3_hero","r4_hero","r5_hero","d1_hero","d2_hero","d3_hero","d4_hero","d5_hero"]

In [647]:
le=LabelEncoder()
data["r1_hero"]=le.fit_transform(data["r1_hero"])
for i in hero:
    data[i]=le.transform(data[i])

In [648]:
#Кодирование мешком слов
X_pick = np.zeros((data.shape[0], 108))
for i, match_id in enumerate(data.index):
    for p in range(5):
        X_pick[i, data.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, data.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1

/Users/roman/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  """


In [649]:
data.drop(["r2_hero","r3_hero","r4_hero","r5_hero","d1_hero","d2_hero","d3_hero","d4_hero","d5_hero"],axis=1,inplace=True)

In [650]:
x=data.iloc[:,:-1]
y=data.iloc[:,-1]
x=scaler.fit_transform(x)

In [651]:
df=np.concatenate((x,X_pick), axis=1)

In [652]:
df.shape

(97230, 201)

#### 5 Проведите кросс-валидацию для логистической регрессии на новой выборке с подбором лучшего параметра регуляризации. Какое получилось качество? Улучшилось ли оно? Чем вы можете это объяснить?

In [653]:
y=y
x=df

In [654]:
lr=LogisticRegression(penalty='l2',random_state=241,C=0.10000000000000001)
C={"C":np.linspace(0.1,10,20)}
grid=GridSearchCV(lr,param_grid=C,scoring='roc_auc',cv=kf)

In [636]:
# Подбор наилучшего С
grid.fit(X=x,y=y)
grid.best_estimator_.C

0.10000000000000001

In [655]:
%time np.average(cross_val_score(lr,X=x,y=y,cv=kf,scoring='roc_auc')) 

CPU times: user 22.9 s, sys: 566 ms, total: 23.4 s
Wall time: 22.8 s


0.75192403913288375

#### 6 Постройте предсказания вероятностей победы команды Radiant для тестовой выборки с помощью лучшей из изученных моделей (лучшей с точки зрения AUC-ROC на кросс-валидации). Убедитесь, что предсказанные вероятности адекватные — находятся на отрезке [0, 1], не совпадают между собой (т.е. что модель не получилась константной).

In [679]:
test=pd.read_csv("features_test.csv")
test.drop(["lobby_type"],axis=1,inplace=True)
test.fillna(value=0,inplace=True)

In [680]:
le=LabelEncoder()
test["r1_hero"]=le.fit_transform(test["r1_hero"])
for i in hero:
    test[i]=le.transform(test[i])

In [681]:
#Кодирование мешком слов
X_pick = np.zeros((test.shape[0], 108))
for i, match_id in enumerate(test.index):
    for p in range(5):
        X_pick[i, test.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, test.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1

/Users/roman/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  """


In [682]:
test.drop(["r2_hero","r3_hero","r4_hero","r5_hero","d1_hero","d2_hero","d3_hero","d4_hero","d5_hero"],axis=1,inplace=True)

In [683]:
x=scaler.fit_transform(test)

In [684]:
dfTest=np.concatenate((x,X_pick), axis=1)

In [685]:
lr=LogisticRegression(penalty='l2',random_state=241,C=0.10000000000000001)

In [686]:
data=pd.read_csv("features.csv")
data.drop(["duration",
           "tower_status_radiant",
           "tower_status_dire",
           "barracks_status_dire",
           "barracks_status_radiant",
           "lobby_type"],axis=1,inplace=True)
data.fillna(value=0,inplace=True)

le=LabelEncoder()
data["r1_hero"]=le.fit_transform(data["r1_hero"])
for i in hero:
    data[i]=le.transform(data[i])
    
#Кодирование мешком слов
X_pick = np.zeros((data.shape[0], 108))
for i, match_id in enumerate(data.index):
    for p in range(5):
        X_pick[i, data.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, data.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1
        
data.drop(["r2_hero","r3_hero","r4_hero","r5_hero","d1_hero","d2_hero","d3_hero","d4_hero","d5_hero"],axis=1,inplace=True)

x=data.iloc[:,:-1]
y=data.iloc[:,-1]
x=scaler.fit_transform(x)

df=np.concatenate((x,X_pick), axis=1)

/Users/roman/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:19: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix


In [687]:
lr.fit(X=df,y=y)

LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=241, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [692]:
ansver=lr.predict_proba(dfTest)[:, 1]

In [696]:
test["match_id"]

0             6
1             7
2            10
3            13
4            16
5            18
6            19
7            24
8            33
9            37
10           41
11           42
12           55
13           60
14           62
15           63
16           64
17           71
18           72
19           83
20           85
21           89
22           92
23          100
24          102
25          108
26          111
27          126
28          130
29          140
          ...  
17147    114165
17148    114168
17149    114171
17150    114183
17151    114188
17152    114192
17153    114195
17154    114202
17155    114209
17156    114211
17157    114217
17158    114232
17159    114236
17160    114238
17161    114242
17162    114247
17163    114285
17164    114286
17165    114314
17166    114327
17167    114330
17168    114337
17169    114348
17170    114349
17171    114361
17172    114369
17173    114377
17174    114378
17175    114393
17176    114398
Name: match_id, Length: 

In [697]:
answer

array([ 0.83341113,  0.76161216,  0.19595967, ...,  0.24660424,
        0.63388196,  0.43678442])

In [702]:
answer=pd.DataFrame(answer)

In [719]:
match_id=pd.DataFrame(test["match_id"])

In [740]:
result=pd.concat([match_id,answer],axis=1)

In [742]:
result.columns=['match_id','radiant_win']

In [746]:
result.to_csv("answer.csv",sep=',',index=False)

### Отчет

#### 1 Какое качество получилось у логистической регрессии над всеми исходными признаками? Как оно соотносится с качеством градиентного бустинга? Чем вы можете объяснить эту разницу? Быстрее ли работает логистическая регрессия по сравнению с градиентным бустингом?

Качество логистической регрессии (с маштабированными признаками и подбором параметра регуляризации) = 0.71630660692837989
Логистическая регресия показывает качество лучше чем градиентный бустинг на 0.026810400869259787
Логистическая регрессия работает существенно быстрее, т.к линейно разделяет гиперпространство

#### 2 Как влияет на качество логистической регрессии удаление категориальных признаков (укажите новое значение метрики качества)? Чем вы можете объяснить это изменение?

Удаление категориальных признаков незначительно улучшило модель (0.71637077592651877) на 6.416899813888044e-05
Так произошло потому что кодирование категориальных признаков числами не лучше решение, для этих целей кодирование мешком слов подходит лучше

#### 3 Сколько различных идентификаторов героев существует в данной игре?

108

#### 4 Какое получилось качество при добавлении "мешка слов" по героям? Улучшилось ли оно по сравнению с предыдущим вариантом? Чем вы можете это объяснить?

Качество при добавлении мешка слов = 0.75187359147615185, что лучше предыдушего результата на 0.03550281554963308
Качество улучшилось т.к категориальная информация о героях (давально важная) была закодированнв с помощью мешка слов, что для алгоритма лучше интерпритируемо.

#### 5 Какое минимальное и максимальное значение прогноза на тестовой выборке получилось у лучшего из алгоритмов?

Max=0.99660683269737305
Min=0.0084920441018697945